In [1]:
# Install necessary libraries
!pip install annoy torchvision



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=a7e6959a4a99e8f0000ae2e5bdb75e1ea9a05ee7024a79eb485e378ffd0ba07c
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [2]:
from annoy import AnnoyIndex
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image


In [3]:
# Define function to get image embeddings
def get_image_embedding(image_path):
    model = models.resnet18(pretrained=True)
    model.eval()  # Set the model to evaluation mode
    preprocess = transforms.Compose([
        transforms.Resize(256),                 # Resize the image to 256x256
        transforms.CenterCrop(224),             # Crop the center 224x224 part of the image
        transforms.ToTensor(),                  # Convert the image to a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet stats
    ])
    image = Image.open(image_path)  # Open the image file
    image_tensor = preprocess(image).unsqueeze(0)  # Preprocess and add batch dimension
    with torch.no_grad():  # Disable gradient calculation
        features = model(image_tensor).numpy()  # Get feature vectors
    return features.flatten()  # Flatten the feature vector


In [6]:
# Sample image paths
image_paths = ["/content/panigalev4.jpeg", "/content/kawasaki.jpeg", "/content/Yamaha_R6.jpeg"]

In [7]:
# Get embeddings for the sample images
embeddings = np.array([get_image_embedding(path) for path in image_paths])


In [8]:
# Create an Annoy index
dimension = embeddings.shape[1]  # Get the embedding dimension
index = AnnoyIndex(dimension, 'angular')  # Use angular distance (cosine similarity)
for i, embedding in enumerate(embeddings):
    index.add_item(i, embedding)  # Add embeddings to the index
index.build(10)  # Build the index with 10 trees

True

In [10]:
# Query: Find similar images to the query image
query_embedding = get_image_embedding("/content/dmonster.jpeg")
nearest_neighbors = index.get_nns_by_vector(query_embedding, 2)  # Find 2 nearest neighbors


In [11]:
# Output the nearest neighbors
print("Nearest neighbors:", nearest_neighbors)


Nearest neighbors: [0, 2]
